In [ ]:
# ============================================================
# Evaluate how well CPI and FX explain each product
# in every country (using the big parquet file)
# ============================================================

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from IPython.display import display

# ---- FILE NAMES (must match your folder) ----
PARQUET_FILE = "Exports-by-branches-of-processing-and-countries-2015-2025.parquet"
CPI_FILE     = "Inflation-Consumer price index.csv"
FX_FILE      = "Exchange-rates_2015-2025.csv"


# ----------------- METRIC ---------------------

def wape(y_true, y_pred):
    """Weighted Absolute Percentage Error in %."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return np.nan
    return np.sum(np.abs(y_true - y_pred)) / denom * 100.0


# ----------------- LOADERS --------------------

def load_branches_all_countries():
    """
    Load the parquet file, keep Fob value, convert Month to datetime,
    and keep numeric DATA.
    """
    # If pyarrow is required on your system:
    # df = pd.read_parquet(PARQUET_FILE, engine="pyarrow")
    df = pd.read_parquet(PARQUET_FILE)
    print("Parquet columns:", df.columns)

    # Only 'Fob value' rows
    df = df[df["Unit"] == "Fob value"].copy()

    # Month like '2015M01' -> '2015-01-01'
    df["date"] = pd.to_datetime(df["Month"].str.replace("M", "-") + "-01")

    # Ensure DATA is numeric
    df["DATA"] = pd.to_numeric(df["DATA"], errors="coerce")

    # Keep relevant columns and drop NaNs
    df = df[["Country", "Branches", "date", "DATA"]].dropna(subset=["DATA"])

    return df


def load_cpi():
    """
    Load CPI file, convert Month to datetime, return monthly index series.
    """
    cpi = pd.read_csv(CPI_FILE)
    print("Inflation columns:", cpi.columns)

    cpi["date"] = pd.to_datetime(cpi["Month"].str.replace("M", "-") + "-01")

    cpi = (
        cpi[["date", "Consumer price index Index"]]
        .rename(columns={"Consumer price index Index": "cpi_index"})
        .set_index("date")
        .sort_index()
    )

    return cpi

def load_fx():
    """
    Load FX file, clean Icelandic decimal-comma format, aggregate to monthly
    average USD rate. Uses python engine + on_bad_lines='skip' to avoid
    parser errors on messy rows.
    """
    fx = pd.read_csv(
        FX_FILE,
        engine="python",   # more tolerant parser
        sep=None,          # auto-detect delimiter
        on_bad_lines="skip"
    )
    print("FX raw columns:", fx.columns)

    # Find the date column (it has a BOM in the name in your file)
    date_col_candidates = [c for c in fx.columns if "Dagsetning" in c]
    if not date_col_candidates:
        raise ValueError("Could not find 'Dagsetning' date column in FX file.")
    date_col = date_col_candidates[0]

    usd_col = "Bandaríkjadalur USD miðgengi"

    # Parse dates
    fx[date_col] = pd.to_datetime(fx[date_col], dayfirst=True, errors="coerce")

    # Clean numeric strings like "1.234,56" -> "1234.56"
    fx[usd_col] = (
        fx[usd_col].astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(".", "", regex=False)   # remove thousand separator
        .str.replace(",", ".", regex=False)  # decimal comma -> dot
    )
    fx[usd_col] = pd.to_numeric(fx[usd_col], errors="coerce")

    fx = fx.dropna(subset=[date_col, usd_col])

    # Aggregate to monthly mean
    fx["month"] = fx[date_col].dt.to_period("M").dt.to_timestamp()

    fx_month = (
        fx.groupby("month")[usd_col]
          .mean()
          .reset_index()
          .rename(columns={"month": "date", usd_col: "usd_fx"})
          .set_index("date")
          .sort_index()
    )

    return fx_month


# ----------------- EVALUATION --------------------

def evaluate_all_countries(min_samples=36):
    """
    For each (Country, Branches), fit simple linear models:
      y ~ CPI, y ~ FX, y ~ (CPI + FX)
    using monthly data where CPI+FX exist.
    Returns a DataFrame with R² and WAPE for each combo.
    """
    branches = load_branches_all_countries()
    cpi = load_cpi()
    fx  = load_fx()

    # Merge CPI + FX on date -> macro feature frame
    macro = cpi.join(fx, how="inner")   # index=date, cols: cpi_index, usd_fx
    print("\nMacro feature frame head:")
    print(macro.head())

    results = []

    combos = [
        ("cpi_only", ["cpi_index"]),
        ("fx_only",  ["usd_fx"]),
        ("both",     ["cpi_index", "usd_fx"]),
    ]

    # Loop over every (country, branch) pair
    for (country, branch), g in branches.groupby(["Country", "Branches"]):
        # Aggregate to monthly Fob sums
        series = (
            g.groupby("date")["DATA"]
             .sum()
             .sort_index()
             .rename("y")
        )

        # Join with macro features
        df_join = pd.concat([series, macro], axis=1).dropna()
        if len(df_join) < min_samples:
            continue  # too little data

        y = df_join["y"].astype(float).values

        for combo_name, feat_cols in combos:
            X = df_join[feat_cols].values

            # Standardize features
            X_mean = X.mean(axis=0, keepdims=True)
            X_std  = X.std(axis=0, keepdims=True)
            X_std[X_std == 0] = 1.0
            Xn = (X - X_mean) / X_std

            # Simple linear regression with Ridge
            model = Ridge(alpha=1.0)
            model.fit(Xn, y)
            y_pred = model.predict(Xn)

            r2_val   = r2_score(y, y_pred)
            wape_val = wape(y, y_pred)

            results.append({
                "country":       country,
                "branch":        branch,
                "feature_combo": combo_name,
                "n_samples":     len(y),
                "r2":            r2_val,
                "wape":          wape_val,
            })

    res_df = pd.DataFrame(results)
    return res_df


# ================== RUN & SUMMARIZE ==================

print("Running evaluation across all countries...\n")
results_all = evaluate_all_countries(min_samples=36)

print("\n===== TOP 20 PRODUCTS BY R² (any country, any combo) =====")
top20 = results_all.sort_values("r2", ascending=False).head(20)
display(top20[["country", "branch", "feature_combo", "n_samples", "r2", "wape"]])

print("\n===== BEST COMBO PER (COUNTRY, BRANCH) (sorted by R²) =====")
best_per = (
    results_all.sort_values("r2", ascending=False)
               .drop_duplicates(subset=["country", "branch"])
)
display(best_per[["country", "branch", "feature_combo", "n_samples", "r2", "wape"]])

# Example: see only United States results
print("\n===== UNITED STATES ONLY (sorted by R²) =====")
us_only = results_all[results_all["country"] == "United States"] \
                     .sort_values("r2", ascending=False)
display(us_only[["country", "branch", "feature_combo", "n_samples", "r2", "wape"]])


Running evaluation across all countries...

Parquet columns: Index(['Branches', 'Country', 'Month', 'Unit', 'DATA'], dtype='object')
Parquet columns: Index(['Branches', 'Country', 'Month', 'Unit', 'DATA'], dtype='object')
Inflation columns: Index(['Month', 'Consumer price index Index',
       'Consumer price index Monthly change, %',
       'Consumer price index Annual change, %'],
      dtype='object')
Inflation columns: Index(['Month', 'Consumer price index Index',
       'Consumer price index Monthly change, %',
       'Consumer price index Annual change, %'],
      dtype='object')


ParserError: Error tokenizing data. C error: Expected 4 fields in line 1332, saw 10
